In [2]:
import pybullet as p
import math
import time
from datetime import datetime

In [3]:
physicsClientID = p.connect(p.GUI)

p.getConnectionInfo(physicsClientID)

import pybullet_data

p.setAdditionalSearchPath(pybullet_data.getDataPath())

robot = 0
numJoints = 0

In [4]:
def import_urdf():
    # plane = p.loadURDF('plane.urdf')
    global robot, numJoints
    StartPos = [0, 0, 3]
    StartOrientation = p.getQuaternionFromEuler([1.57, 0, 0])
    robot = p.loadURDF("humanoid.urdf", StartPos, StartOrientation, useFixedBase=1)
    position, orientation = p.getBasePositionAndOrientation(robot)
    numBodies = p.getNumBodies()

    numJoints = p.getNumJoints(robot)
    p.setGravity(0, 0, 0)
    
def joint_details():
    for i in range(numJoints):
        if p.getJointInfo(robot, i)[2] == 0: #only details of the revolute joints
            print(p.getJointInfo(robot, i)[0], str(p.getJointInfo(robot, i)[1]).lstrip('b'))

# *** ALTERNATIVE APPROACH *** (not working)

# def ANGLE_limits(jointIndex, bodyID):
#     lowerLimit = p.getJointInfo(bodyID, jointIndex)[9]
#     upperLimit = p.getJointInfo(bodyID, jointIndex)[8]

#     return lowerLimit, upperLimit

# def targetAngle(lowerLimit, upperLimit, timePeriod, t):
#     Angle = (upperLimit - lowerLimit) * abs(math.sin( (2 * math.pi * t) / timePeriod)) + lowerLimit

#     return Angle

# def joint_control_pos_sin(joint_index, timePeriod = 5, bodyID=robot):

#     t=0

#     maxForce = 0.01

#     lowerLimit, upperLimit = ANGLE_limits(joint_index, bodyID)

#     p.setRealTimeSimulation(1)

#     while True:
#         target_Angle = targetAngle(lowerLimit, upperLimit, timePeriod, t)

#         p.setJointMotorControl2(bodyID, joint_index, p.POSITION_CONTROL,  target_Angle, maxForce)

#         try:
#             timeStep = p.getJointState(bodyID, joint_index)[1] / abs((p.getJointState(bodyID, joint_index)[0] - target_Angle))
#         except ZeroDivisionError:
#             timeStep = 1.41423782742495e-14

#         print(timeStep)
        
#         time.sleep(timeStep)

#         t += timeStep


# sin controller functions

def ANGLE_limits(jointIndex, bodyID):
    lowerLimit = p.getJointInfo(bodyID, jointIndex)[9]
    upperLimit = p.getJointInfo(bodyID, jointIndex)[8]

    return lowerLimit, upperLimit


def targetAngle(lowerLimit, upperLimit, timePeriod, t):
    Angle = (upperLimit - lowerLimit) * \
        abs(math.sin((math.pi * t) / timePeriod)) + lowerLimit

    return Angle


def joint_control_pos_sin_2(joint_index, timePeriod=30, bodyID=robot):

    t = 0

    maxForce = 0.02

    lowerLimit, upperLimit = ANGLE_limits(joint_index, bodyID)

    p.setRealTimeSimulation(1)
    i = 0
    while t <= timePeriod:

        # time.sleep(0.01)

        target_Angle = targetAngle(lowerLimit, upperLimit, timePeriod, t)

        current_angle = p.getJointState(bodyID, joint_index)[0]
        print(target_Angle, current_angle)

        before = datetime.timestamp(datetime.now())

        p.setJointMotorControl2(bodyID, joint_index,
                                p.POSITION_CONTROL, target_Angle, maxForce)

        while True:

            time.sleep(1e-04)
            current_angle = float(p.getJointState(bodyID, joint_index)[0])

            if (current_angle >= min(target_Angle-0.001, target_Angle+0.001) and current_angle <= max(target_Angle-0.001, target_Angle+0.001)):
                break

        after = datetime.timestamp(datetime.now())
        dynamic_timestep = after - before
        print(after - before)
        print()
        print(t)
        print()
        # t += 0.25 #if constant time step is required. But this value is based on system perforamnce
        t += dynamic_timestep
        i = 2


In [5]:
import_urdf()

In [6]:
time.sleep(2)
joint_control_pos_sin_2(10)

0.0 0.0
0.013996124267578125

0

-0.004602204368869054 0.0
0.12299990653991699

0.013996124267578125

-0.045045493910800614 -0.003609104226949448
0.22100114822387695

0.13699603080749512

-0.11768907126264015 -0.044074865731059336
0.2491621971130371

0.35799717903137207

-0.1995116766066046 -0.11673802875002752
0.2490251064300537

0.6071593761444092

-0.28115356441626266 -0.19853109568409474
0.246002197265625

0.8561844825744629

-0.3616169020707573 -0.2801569539071634
0.25200319290161133

1.102186679840088

-0.44379379519145684 -0.3606531705658565
0.2510089874267578

1.3541898727416992

-0.5253393367371276 -0.4428273455129343
0.2520599365234375

1.605198860168457

-0.6068609863946088 -0.5243739037532326
0.25256896018981934

1.8572587966918945

-0.6881231514928456 -0.6058955936955958
0.24700093269348145

2.109827756881714

-0.7671287014800778 -0.6871435518997971
0.24901103973388672

2.3568286895751953

-0.8462575627207429 -0.7661673551805972
0.2510230541229248

2.605839729309082

-0.92

In [16]:
p.setRealTimeSimulation(0)
p.resetSimulation()
# go to urdf loading cell

In [17]:
import_urdf()